In [2]:
# Exercise 1: Use bulk_write to insert, update, and delete multiple documents simultaneously.
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Hapus semua data sebelumnya agar hasil bersih
courses_collection.delete_many({})

# Definisi operasi bulk (Insert, Update, Delete)
operations = [
    # INSERT: Tambahkan beberapa data courses
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),

    # UPDATE: Ubah jumlah enrollments pada dua course
    UpdateOne({'course': 'Math 101'}, {'$set': {'enrollments': 35}}),
    UpdateOne({'course': 'CS 102'}, {'$inc': {'enrollments': 5}}), 

    # DELETE: Hapus satu course
    DeleteOne({'course': 'History 201'})
]

# Eksekusi semua operasi secara bersamaan
result = courses_collection.bulk_write(operations)

# Tampilkan hasil ringkas
print("Bulk operations executed successfully.\n")
print(f"Total Inserted : {result.inserted_count}")
print(f"Total Modified : {result.modified_count}")
print(f"Total Deleted  : {result.deleted_count}\n")

# Menampilkan seluruh data setelah operasi
print("Data Courses Setelah Operasi:")
for doc in courses_collection.find():
    print(doc)

Courses inserted successfully.
Total inserted: 4 documents

Daftar Data Courses:
{'_id': ObjectId('6915a09562ccbe74b55dfc29'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('6915a09562ccbe74b55dfc2a'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('6915a09562ccbe74b55dfc2b'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}
{'_id': ObjectId('6915a09562ccbe74b55dfc2c'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}


In [3]:
# Task 1: Insert a dataset of courses where each course has multiple student enrollments, then display the data.
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Bersihkan koleksi agar tidak duplikat data sebelumnya
courses_collection.delete_many({})

# Operasi bulk insert — menambahkan beberapa dokumen sekaligus
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]

# Eksekusi operasi bulk_write
result = courses_collection.bulk_write(operations)
print('Courses inserted successfully.')
print(f"Total inserted: {result.inserted_count} documents\n")

# Menampilkan seluruh data dari koleksi
print("Daftar Data Courses:")
for course in courses_collection.find():
    print(course)



Bulk operations executed successfully.

Total Inserted : 4
Total Modified : 2
Total Deleted  : 1

Data Courses Setelah Operasi:
{'_id': ObjectId('6915a18762ccbe74b55dfc2e'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('6915a18762ccbe74b55dfc2f'), 'course': 'CS 102', 'enrollments': 30, 'department': 'Computer Science'}
{'_id': ObjectId('6915a18762ccbe74b55dfc31'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}


In [ ]:
# Exercise 2: Filter courses with enrollments over 20 students.

# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)


In [ ]:
# Task 2: Retrieve and display courses only from the 'Computer Science' or 'Mathematics' departments.

# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

In [ ]:
# Exercise 3: Use aggregation to find the average enrollment per department

# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

In [ ]:
# Task 3: Create an aggregation pipeline that finds the maximum enrollment for each department.

# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

In [ ]:
# Exercise 4: Use $project to rename and only show fields: course_name, department_name, and enrollments.

# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

In [ ]:
# Task 4: Use $addFields to create a new field enrollment_category where enrollments > 20 are 'high' and others 'low'.

# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

In [5]:
# Homework 1: Perform an aggregation to get a count of courses per department.

pipeline = [
    {
        '$group': {
            '_id': '$department',       
            'total_courses': {'$sum': 1}  
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Mathematics', 'total_courses': 1}
{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'Physics', 'total_courses': 1}


In [6]:
# Homework 2: Use $match and $group together to filter and get only courses with enrollments over 25 in 'Computer Science'

pipeline = [
    {
        '$match': {
            'department': 'Computer Science',  
            'enrollments': {'$gt': 25}         
        }
    },
    {
        '$group': {
            '_id': '$department',              
            'total_courses': {'$sum': 1},      
            'average_enrollment': {'$avg': '$enrollments'}  
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'total_courses': 1, 'average_enrollment': 30.0}


In [4]:
# Homework 3: Apply $lookup to join courses collection with students collection based on student enrollments.

pipeline = [
    {
        '$lookup': {
            'from': 'students',         
            'localField': 'enrollments', 
            'foreignField': 'enrollments', 
            'as': 'student_details'     
        }
    },
    {
        '$project': {                    
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'student_details.name': 1,   
            'student_details.id': 1
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'student_details': []}
{'course': 'CS 102', 'enrollments': 30, 'department': 'Computer Science', 'student_details': []}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'student_details': []}
